In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

### Step 1: Data Preparation


In [ ]:
# Create DataFrame
data = pd.read_excel("IMB469-XLS-ENG (2).xlsx")

In [ ]:
data.head()

In [ ]:
data = data.drop(
    columns=["Agmt No", "Contract Status", "Start_Date", "DATASET", "DefaulterFlag"]
)

In [ ]:
numeric_features = [
    "AGE",
    "NOOFDEPE",
    "MTHINCTH",
    "SALDATFR",
    "TENORYR",
    "DWNPMFR",
]

In [ ]:
categorical_features = [
    "Region",
    "Branch",
]

In [ ]:
# Split dataset into training and testing sets
train_data, test_data = train_test_split(
    data, test_size=0.2, random_state=42, stratify=data["DefaulterType"]
)

### Step 2: Feature Scaling


In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the scaler on numeric features
train_data_scaled = train_data.copy()
train_data_scaled[numeric_features] = scaler.fit_transform(train_data[numeric_features])

In [ ]:
# Fit and transform the scaler on numeric features
test_data_scaled = test_data.copy()
test_data_scaled[numeric_features] = scaler.transform(test_data[numeric_features])

In [ ]:
# Convert categorical features to one-hot encoded columns
one_hot_encoder = OneHotEncoder()

encoded_data = one_hot_encoder.fit_transform(train_data_scaled[categorical_features])

# Convert the encoded data to a DataFrame
train_data_encoded_categorical = pd.DataFrame(
    encoded_data.toarray(),
    columns=one_hot_encoder.get_feature_names_out(categorical_features),
)

In [ ]:
train_data_scaled_without_categorical = train_data_scaled.drop(
    columns=categorical_features
).reset_index(drop=True)
train_data_encoded = pd.concat(
    [train_data_scaled_without_categorical, train_data_encoded_categorical], axis=1
)

In [ ]:
# Separate features and target variable
y_train = train_data_encoded["DefaulterType"]
X_train = train_data_encoded.drop(["DefaulterType"], axis=1)

In [ ]:
# Convert categorical features to one-hot encoded columns
test_encoded_data = one_hot_encoder.transform(test_data_scaled[categorical_features])

test_data_encoded_categorical = pd.DataFrame(
    test_encoded_data.toarray(),
    columns=one_hot_encoder.get_feature_names_out(categorical_features),
)

test_data_scaled_without_categorical = test_data_scaled.drop(
    columns=categorical_features
).reset_index(drop=True)
test_data_encoded = pd.concat(
    [test_data_scaled_without_categorical, test_data_encoded_categorical], axis=1
)

In [ ]:
# Prepare test data
y_test = test_data_encoded["DefaulterType"]
X_test = test_data_encoded.drop(["DefaulterType"], axis=1)

### Step 3: Model Training


In [ ]:
pd.options.display.max_columns = 36

In [ ]:
X_train.head()

In [ ]:
y_train

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


# Initialize models
logreg_model = LogisticRegression()
tree_model = DecisionTreeClassifier()
rf_model = RandomForestClassifier()
svm_model = SVC()

# Train models
logreg_model.fit(X_train, y_train)
tree_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)
svm_model.fit(X_train, y_train)

### Step 4: Model Evaluation


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Make predictions
logreg_pred = logreg_model.predict(X_test)
tree_pred = tree_model.predict(X_test)
rf_pred = rf_model.predict(X_test)
svm_pred = svm_model.predict(X_test)

# Evaluate models
logreg_accuracy = accuracy_score(y_test, logreg_pred)
tree_accuracy = accuracy_score(y_test, tree_pred)
rf_accuracy = accuracy_score(y_test, rf_pred)
svm_accuracy = accuracy_score(y_test, svm_pred)

logreg_precision = precision_score(y_test, logreg_pred, average="micro")
tree_precision = precision_score(y_test, tree_pred, average="micro")
rf_precision = precision_score(y_test, rf_pred, average="micro")
svm_precision = precision_score(y_test, svm_pred, average="micro")

logreg_recall = recall_score(y_test, logreg_pred, average="micro")
tree_recall = recall_score(y_test, tree_pred, average="micro")
rf_recall = recall_score(y_test, rf_pred, average="micro")
svm_recall = recall_score(y_test, svm_pred, average="micro")

logreg_f1 = f1_score(y_test, logreg_pred, average="micro")
tree_f1 = f1_score(y_test, tree_pred, average="micro")
rf_f1 = f1_score(y_test, rf_pred, average="micro")
svm_f1 = f1_score(y_test, svm_pred, average="micro")

In [ ]:
print(
    f"logreg_recall: {logreg_recall}, tree_recall: {tree_recall}, rf_recall: {rf_recall}, svm_recall: {svm_recall}"
)

In [ ]:
print(
    f"logreg_accuracy: {logreg_accuracy}, tree_accuracy: {tree_accuracy}, rf_accuracy: {rf_accuracy}, svm_accuracy: {svm_accuracy}"
)

### Step 5: Interpretability

Explain the decision rules forthe models using their .feature*importances* and .coef\_ attributes.


In [ ]:
import matplotlib.pyplot as plt

# Get feature importances from the models
tree_feature_importances = tree_model.feature_importances_
rf_feature_importances = rf_model.feature_importances_

# Get feature names
feature_names = X_train.columns

# Plot feature importances
plt.figure(figsize=(10, 6))
plt.barh(
    range(len(feature_names)),
    tree_feature_importances,
    align="center",
    label="Decision Tree",
)
plt.barh(
    range(len(feature_names)),
    rf_feature_importances,
    align="center",
    label="Random Forest",
    alpha=0.6,
)
plt.yticks(range(len(feature_names)), feature_names)
plt.xlabel("Feature Importance")
plt.ylabel("Feature")
plt.title("Feature Importance from Decision Tree and Random Forest")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Get absolute coefficients from the logistic regression model
logreg_coeffs = np.exp(logreg_model.coef_[0])

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(range(len(feature_names)), logreg_coeffs, align="center")
plt.yticks(range(len(feature_names)), feature_names)
plt.xlabel("Coefficient")
plt.ylabel("Feature")
plt.title("Feature Importance from Logistic Regression")
plt.tight_layout()
plt.show()

### Step 6: Parameter Tuning

You can use techniques like grid search or random search to find optimal hyperparameters for each algorithm


Parameter tuning is a crucial step in optimizing the performance of machine learning models. Each algorithm has various parameters that can be adjusted to improve their predictive accuracy and generalization. In this step, you search for the best combination of parameter values to achieve the best model performance. Let's go through an explanation and example for each algorithm.


1. Logistic Regression:

For logistic regression, one common parameter is the regularization strength (C). A smaller C increases the regularization, which can prevent overfitting.


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {"C": [0.01, 0.1, 1, 10, 100]}

# Initialize GridSearchCV
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring="recall")

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameter
best_C = grid_search.best_params_["C"]

# Train a Logistic Regression model with the best parameter
best_logreg_model = LogisticRegression(C=best_C)
best_logreg_model.fit(X_train, y_train)

y_pred = best_logreg_model.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the best C value and metrics
print(f"Best C parameter: {best_C}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

2. Decision Trees:

For decision trees, an important parameter is the maximum depth of the tree (max_depth). A deeper tree can lead to overfitting, so you need to find an optimal value for this parameter.


In [ ]:
param_grid = {"max_depth": [None, 5, 10, 15, 20]}

grid_search = GridSearchCV(
    DecisionTreeClassifier(), param_grid, cv=5, scoring="recall"
)

grid_search.fit(X_train, y_train)

best_max_depth = grid_search.best_params_["max_depth"]

best_tree_model = DecisionTreeClassifier(max_depth=best_max_depth)
best_tree_model.fit(X_train, y_train)

3. Random Forest:

Random Forests have parameters like the number of trees (n_estimators) and the maximum depth of each tree (max_depth). Tuning these parameters can improve the performance of the ensemble.


In [ ]:
param_grid = {"n_estimators": [50, 100, 150], "max_depth": [None, 10, 20]}

grid_search = GridSearchCV(
    RandomForestClassifier(), param_grid, cv=5, scoring="recall"
)

grid_search.fit(X_train, y_train)

best_n_estimators = grid_search.best_params_["n_estimators"]
best_max_depth = grid_search.best_params_["max_depth"]

best_rf_model = RandomForestClassifier(
    n_estimators=best_n_estimators, max_depth=best_max_depth
)
best_rf_model.fit(X_train, y_train)

4. Support Vector Machines (SVM):

For SVMs, parameters like the regularization parameter (C) and the choice of kernel (kernel) are important to tune. The kernel can be linear, polynomial, or radial basis function (RBF).


In [ ]:
param_grid = {"C": [0.1, 1, 10], "kernel": ["linear", "poly", "rbf"]}

grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring="recall")

grid_search.fit(X_train, y_train)

best_C = grid_search.best_params_["C"]
best_kernel = grid_search.best_params_["kernel"]

best_svm_model = SVC(C=best_C, kernel=best_kernel)
best_svm_model.fit(X_train, y_train)

In each example, we're using GridSearchCV to perform a grid search over the specified parameter values. The cv parameter indicates the number of folds in cross-validation, and scoring specifies the evaluation metric. After finding the best parameter combination, a new model is trained using the best parameters.

Remember that these are simplified examples, and you can explore more parameters and values to fine-tune your models further. Also, be mindful of the computational cost of exhaustive grid searches, as they can become time-consuming with larger datasets or complex models.


## Step 7: Conclusion

Summarize the results based on accuracy, precision, recall, and F1-score metrics and discuss which algorithm might be the most suitable for this loan approval prediction problem.
